<a href="https://colab.research.google.com/github/Shridharpawar77/Llama_3.2-vl_model_finetuning/blob/main/llama3_2_vision_model_finetuning_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "torch==2.4.0" tensorboard pillow torchvision accelerate huggingface_hub
!pip install  --upgrade \
  "transformers==4.45.1" \
  "datasets==3.0.1" \
  "accelerate==0.34.2" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.44.0" \
  "trl==0.11.1" \
  "peft==0.13.0" \
  "qwen_vl_utils"

In [3]:
from huggingface_hub import login

login(
  token="", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [ ]:
!pip install trl --upgrade

In [10]:
from PIL import Image

In [11]:
import os
import pandas as pd
from PIL import Image
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# --------- your config ----------
CSV_PATH = "/content/output.csv"            # your CSV
IMAGE_ROOT_DIR = "/content/sample_data"     # base dir for images
MODEL_NAME = "meta-llama/Llama-3.2-11B-Vision-Instruct"
OUTPUT_DIR = "llama32_vision_collections_lora"
TEST_SPLIT = 0.2
# --------------------------------

# 1) Load CSV
df = pd.read_csv(CSV_PATH)

# 2) Helper to load image from relative path
def load_image(image_rel_path: str):
    image_path = os.path.join(IMAGE_ROOT_DIR, image_rel_path)
    img = Image.open(image_path).convert("RGB")
    return img

# 3) Convert one row -> messages format
def row_to_example(row):
    """
    Expects CSV columns:
      - image_path
      - instruction
      - description
    """
    return {
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": row["instruction"],
                    },
                    {
                        "type": "image",
                        "image": load_image(row["image_path"]),
                    },
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {
                        "type": "text",
                        "text": row["description"],
                    }
                ],
            },
        ]
    }

# 4) Build list of training examples
dataset = [row_to_example(row) for _, row in df.iterrows()]

# Optional: quick sanity check (similar to your print(dataset[345]["messages"]))
print(dataset[0]["messages"])


[{'role': 'user', 'content': [{'type': 'text', 'text': 'Describe the image in details and try to analyse the financial situation of the property/surrounding/person appearing in the image'}, {'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=462x280 at 0x7B8050F92F30>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'The home is built using traditional materials such as clay bricks and clay. The home appears to be in closed state as doors are closed, the home surrounding appears to be well maintained but overall there seems to be financial distress. The loan lenders should opt for loan settlement.'}]}]


In [12]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig

# Hugging Face model id
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",
    # attn_implementation="flash_attention_2", # not supported for training
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/89.4k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

...

preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.09k [00:00<?, ?B/s]

In [13]:
text = processor.apply_chat_template(
    dataset[2]["messages"], tokenize=False, add_generation_prompt=False
)
text

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nDescribe the image in details and try to analyse the financial situation of the property/surrounding/person appearing in the image<|image|><|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nthe image shows outside area of a home and there is a bike parked outside with a cycle and some cloths hanging, the home conditions looks moderate and one can gess of less financial distrace as assets such as bike cycle are visiable outside the home. there seems to be less financial distress. The loan lenders should allow some more time for this lender.<|eot_id|>'

In [14]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)


In [19]:
from trl import SFTConfig


args = SFTConfig(
    output_dir="fine-tuned-visionllama", # directory to save and repository id
    num_train_epochs=5,                     # number of training epochs
    per_device_train_batch_size=2,          # batch size per device during training
    gradient_accumulation_steps=4,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=4,                        # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=False,                      # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    gradient_checkpointing_kwargs = {"use_reentrant": False}, # use reentrant checkpointing
    dataset_text_field="", # need a dummy field for collator
    dataset_kwargs = {"skip_prepare_dataset": True} # important for collator
)
args.remove_unused_columns=False

In [20]:
from transformers import Qwen2VLProcessor
from qwen_vl_utils import process_vision_info

def collate_fn(examples):
    # Get the texts and images, and apply the chat template
    texts = [processor.apply_chat_template(example["messages"], tokenize=False) for example in examples]
    image_inputs = [process_vision_info(example["messages"])[0] for example in examples]

    # Tokenize the texts and process the images
    batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)

    # The labels are the input_ids, and we mask the padding tokens in the loss computation
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100  #
    # Ignore the image token index in the loss computation (model specific)
    if isinstance(processor, Qwen2VLProcessor):
        image_tokens = [151652,151653,151655]
    else:
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100
    batch["labels"] = labels

    return batch

In [21]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    data_collator=collate_fn,
    peft_config=peft_config
)

In [22]:
# trainer.args.per_device_train_batch_size = 1 # Reduce batch size to avoid OOM
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Step,Training Loss
4,3.581300
8,3.163000


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/usr/local/lib/python3.12/dist-packages/torch/uti

TrainOutput(global_step=10, training_loss=3.2146017074584963, metrics={'train_runtime': 74.1836, 'train_samples_per_second': 0.674, 'train_steps_per_second': 0.135, 'total_flos': 353214920126532.0, 'train_loss': 3.2146017074584963, 'entropy': 2.738122081756592, 'num_tokens': 5285.0, 'mean_token_accuracy': 0.47432404160499575, 'epoch': 5.0})